In [19]:
import time

notebook_start_time = time.time()

# Setup Environment

In [20]:
import sys
from pathlib import Path

def is_google_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False
    
if is_google_colab():
    print("Running in Google Colab")
else:
    print("Not running in Google Colab")
    root_dir = str(Path().absolute().parent)

if root_dir not in sys.path:
    print(f"Adding {root_dir} to sys.path")
    sys.path.append(root_dir)

Not running in Google Colab


# Feature Pipeline

## Imports

In [28]:
%load_ext autoreload
%autoreload 2

import warnings
from pprint import pprint

import polars as pl
import torch
from loguru import logger
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")

from recsys.raw_data_sources import myanimelist_dataset
from recsys.features.anime import (compute_features_of_anime)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## MyAnimeList Dataset

### Anime Data

- MAL_ID - MyAnimelist ID of the anime.

In [50]:
anime_df = myanimelist_dataset.extract_anime_data()
anime_df.shape

(16214, 36)

In [51]:
anime_df.head(3)

MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Synopsis
i64,str,f64,str,str,str,str,i64,str,str,str,str,str,str,str,str,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
1,"""Cowboy Bebop""",8.78,"""Action, Adventure, Comedy, Dra…","""Cowboy Bebop""","""カウボーイビバップ""","""TV""",26,"""Apr 3, 1998 to Apr 24, 1999""","""Spring 1998""","""Bandai Visual""","""Funimation, Bandai Entertainme…","""Sunrise""","""Original""","""24 min. per ep.""","""R - 17+ (violence & profanity)""",28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,"""In the year 2071, humanity has…"
5,"""Cowboy Bebop: Tengoku no Tobir…",8.39,"""Action, Drama, Mystery, Sci-Fi…","""Cowboy Bebop:The Movie""","""カウボーイビバップ 天国の扉""","""Movie""",1,"""Sep 1, 2001""",null,"""Sunrise, Bandai Visual""","""Sony Pictures Entertainment""","""Bones""","""Original""","""1 hr. 55 min.""","""R - 17+ (violence & profanity)""",159.0,518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0,"""other day, another bounty—such…"
6,"""Trigun""",8.24,"""Action, Sci-Fi, Adventure, Com…","""Trigun""","""トライガン""","""TV""",26,"""Apr 1, 1998 to Sep 30, 1998""","""Spring 1998""","""Victor Entertainment""","""Funimation, Geneon Entertainme…","""Madhouse""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",266.0,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0,"""Vash the Stampede is the man w…"


In [52]:
anime_df = compute_features_of_anime(anime_df)
anime_df.shape


(10899, 38)

#### Create an embedding for each anime description

In [53]:
for i, desc in enumerate(anime_df["description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

2025-04-12 19:45:36.689 | INFO     | __main__:<module>:2 - Item 1:
Cowboy Bebop is a TV anime with a rating of 8.78/10.
It has 26 episodes and was released in Apr 3, 1998 to Apr 24, 1999. It is rated R - 17+ (violence & profanity) and it is ranked 39, of Most Popular Anime.
It belongs to the Action, Adventure, Comedy, Drama, Sci-Fi, Space genres.
It has a synopsis of In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered